## Test train model with completely new simulate transaction

### Load Super Learner Model

In [1]:
import os
import sys
import joblib
import numpy as np

sys.path.append('./saved_models')
sys.path.append('./utils')

import transform_data_functions as utils_transform
import ml_training_functions as utils_training
import file_handler_functions as utils_file
import performance_assessment_function as utils_assessment

FOLDER_PATH = "./saved_models"
FILE_PATH = FOLDER_PATH + "/super_learner_model.pkl"
FILE_PATH_iso_forest = FOLDER_PATH + "/super_learner_model_iso_forest.pkl"

if not os.path.isdir(FOLDER_PATH):
    raise NotADirectoryError(f"Error: The folder '{FOLDER_PATH}' does not exist.")

if not os.path.isfile(FILE_PATH):
    raise FileNotFoundError(f"Error: The file '{FILE_PATH}' does not exist.")

if not os.path.isfile(FILE_PATH_iso_forest):
    raise FileNotFoundError(f"Error: The file '{FILE_PATH_iso_forest}' does not exist.")

load_model = joblib.load(FILE_PATH)
load_model_iso_forest = joblib.load(FILE_PATH_iso_forest)

In [2]:
load_model.keys()

dict_keys(['base_models', 'meta_model', 'scaler', 'input_features'])

In [3]:
load_model_iso_forest.keys()

dict_keys(['base_models', 'meta_model', 'scaler', 'isolation_forest', 'input_features', 'model_type', 'n_base_models', 'feature_count'])

### Load Test set

In [4]:
DIR_INPUT='./data/simulated-data-raw-TEST/' 

BEGIN_DATE = "2019-01-01"
END_DATE = "2019-01-10"

print("Load  files")
%time raw_test_transactions_df = utils_file.read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE)

Load  files
CPU times: user 5.27 ms, sys: 1.84 ms, total: 7.1 ms
Wall time: 7.74 ms


In [5]:
raw_test_transactions_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,0,2018-04-01 00:35:35,25,79,125.33,2135,0,0,0
1,1,2018-04-01 00:51:53,14,669,76.52,3113,0,0,0
2,2,2018-04-01 01:11:00,8,600,5.85,4260,0,0,0
3,3,2018-04-01 01:23:05,15,162,109.77,4985,0,0,0
4,4,2018-04-01 02:25:01,42,348,59.83,8701,0,0,0


### Preprocessing Test set

In [6]:
test_transactions_df = utils_transform.transform_new_df(raw_test_transactions_df)
test_transactions_df_hide_TX_Fraud = utils_transform.transform_new_df(raw_test_transactions_df, isTestSet = True, hideTXFraud = True)

In [7]:
test_transactions_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
0,0,2018-04-01 00:35:35,25,79,125.33,2135,0,0,0,1,...,1.0,125.33,1.0,125.33,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2018-04-01 00:51:53,14,669,76.52,3113,0,0,0,1,...,1.0,76.52,1.0,76.52,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2018-04-01 01:11:00,8,600,5.85,4260,0,0,0,1,...,1.0,5.85,1.0,5.85,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2018-04-01 01:23:05,15,162,109.77,4985,0,0,0,1,...,1.0,109.77,1.0,109.77,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2018-04-01 02:25:01,42,348,59.83,8701,0,0,0,1,...,1.0,59.83,1.0,59.83,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
test_transactions_df_hide_TX_Fraud.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DURING_WEEKEND,...,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TRUE_FRAUD,TRUE_FRAUD_SCENARIO,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
0,0,2018-04-01 00:35:35,25,79,125.33,2135,0,0,0,1,...,1.0,125.33,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2018-04-01 00:51:53,14,669,76.52,3113,0,0,0,1,...,1.0,76.52,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2018-04-01 01:11:00,8,600,5.85,4260,0,0,0,1,...,1.0,5.85,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2018-04-01 01:23:05,15,162,109.77,4985,0,0,0,1,...,1.0,109.77,0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2018-04-01 02:25:01,42,348,59.83,8701,0,0,0,1,...,1.0,59.83,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
test_transactions_df.columns

Index(['TRANSACTION_ID', 'TX_DATETIME', 'CUSTOMER_ID', 'TERMINAL_ID',
       'TX_AMOUNT', 'TX_TIME_SECONDS', 'TX_TIME_DAYS', 'TX_FRAUD',
       'TX_FRAUD_SCENARIO', 'TX_DURING_WEEKEND', 'TX_DURING_NIGHT',
       'CUSTOMER_ID_NB_TX_1DAY_WINDOW', 'CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW',
       'CUSTOMER_ID_NB_TX_7DAY_WINDOW', 'CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW',
       'CUSTOMER_ID_NB_TX_30DAY_WINDOW', 'CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW',
       'TERMINAL_ID_NB_TX_1DAY_WINDOW', 'TERMINAL_ID_RISK_1DAY_WINDOW',
       'TERMINAL_ID_NB_TX_7DAY_WINDOW', 'TERMINAL_ID_RISK_7DAY_WINDOW',
       'TERMINAL_ID_NB_TX_30DAY_WINDOW', 'TERMINAL_ID_RISK_30DAY_WINDOW'],
      dtype='object')

In [10]:
test_transactions_df_hide_TX_Fraud.columns

Index(['TRANSACTION_ID', 'TX_DATETIME', 'CUSTOMER_ID', 'TERMINAL_ID',
       'TX_AMOUNT', 'TX_TIME_SECONDS', 'TX_TIME_DAYS', 'TX_FRAUD',
       'TX_FRAUD_SCENARIO', 'TX_DURING_WEEKEND', 'TX_DURING_NIGHT',
       'CUSTOMER_ID_NB_TX_1DAY_WINDOW', 'CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW',
       'CUSTOMER_ID_NB_TX_7DAY_WINDOW', 'CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW',
       'CUSTOMER_ID_NB_TX_30DAY_WINDOW', 'CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW',
       'TRUE_FRAUD', 'TRUE_FRAUD_SCENARIO', 'TERMINAL_ID_NB_TX_1DAY_WINDOW',
       'TERMINAL_ID_RISK_1DAY_WINDOW', 'TERMINAL_ID_NB_TX_7DAY_WINDOW',
       'TERMINAL_ID_RISK_7DAY_WINDOW', 'TERMINAL_ID_NB_TX_30DAY_WINDOW',
       'TERMINAL_ID_RISK_30DAY_WINDOW'],
      dtype='object')

In [11]:
output_feature = "TX_FRAUD"
output_feature_hide_TX_Fraud = "TRUE_FRAUD"

input_features = load_model['input_features']
print(f"Total input features expected: {len(input_features)}")
print(f"Input features: {input_features}")

X_test = test_transactions_df[input_features]
y_test = test_transactions_df[output_feature]

X_test_hide_TX_Fraud = test_transactions_df_hide_TX_Fraud[input_features]
y_test_hide_TX_Fraud = test_transactions_df_hide_TX_Fraud[output_feature_hide_TX_Fraud]

print(f"\nTest set features shape: {X_test.shape}")
print(f"Test set features (hidden fraud) shape: {X_test_hide_TX_Fraud.shape}")
print(f"\nFeatures present in test_transactions_df_hide_TX_Fraud:")
print(test_transactions_df_hide_TX_Fraud.columns.tolist())

X_test[:10]

Total input features expected: 15
Input features: ['TX_AMOUNT', 'TX_DURING_WEEKEND', 'TX_DURING_NIGHT', 'CUSTOMER_ID_NB_TX_1DAY_WINDOW', 'CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW', 'CUSTOMER_ID_NB_TX_7DAY_WINDOW', 'CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW', 'CUSTOMER_ID_NB_TX_30DAY_WINDOW', 'CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW', 'TERMINAL_ID_NB_TX_1DAY_WINDOW', 'TERMINAL_ID_RISK_1DAY_WINDOW', 'TERMINAL_ID_NB_TX_7DAY_WINDOW', 'TERMINAL_ID_RISK_7DAY_WINDOW', 'TERMINAL_ID_NB_TX_30DAY_WINDOW', 'TERMINAL_ID_RISK_30DAY_WINDOW']

Test set features shape: (1025, 15)
Test set features (hidden fraud) shape: (1025, 15)

Features present in test_transactions_df_hide_TX_Fraud:
['TRANSACTION_ID', 'TX_DATETIME', 'CUSTOMER_ID', 'TERMINAL_ID', 'TX_AMOUNT', 'TX_TIME_SECONDS', 'TX_TIME_DAYS', 'TX_FRAUD', 'TX_FRAUD_SCENARIO', 'TX_DURING_WEEKEND', 'TX_DURING_NIGHT', 'CUSTOMER_ID_NB_TX_1DAY_WINDOW', 'CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW', 'CUSTOMER_ID_NB_TX_7DAY_WINDOW', 'CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW', 'CUSTOMER_ID

,TX_AMOUNT,TX_DURING_WEEKEND,TX_DURING_NIGHT,CUSTOMER_ID_NB_TX_1DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
0,125.33,1,1,1.0,125.330,1.0,125.330,1.0,125.330,0.0,0.0,0.0,0.0,0.0,0.0
1,76.52,1,1,1.0,76.520,1.0,76.520,1.0,76.520,0.0,0.0,0.0,0.0,0.0,0.0
2,5.85,1,1,1.0,5.850,1.0,5.850,1.0,5.850,0.0,0.0,0.0,0.0,0.0,0.0
3,109.77,1,1,1.0,109.770,1.0,109.770,1.0,109.770,0.0,0.0,0.0,0.0,0.0,0.0
4,59.83,1,1,1.0,59.830,1.0,59.830,1.0,59.830,0.0,0.0,0.0,0.0,0.0,0.0
5,78.89,1,1,2.0,77.705,2.0,77.705,2.0,77.705,0.0,0.0,0.0,0.0,0.0,0.0
6,8.75,1,1,1.0,8.750,1.0,8.750,1.0,8.750,0.0,0.0,0.0,0.0,0.0,0.0
7,14.57,1,1,1.0,14.570,1.0,14.570,1.0,14.570,0.0,0.0,0.0,0.0,0.0,0.0
8,55.51,1,1,2.0,82.640,2.0,82.640,2.0,82.640,0.0,0.0,0.0,0.0,0.0,0.0
9,66.09,1,1,1.0,66.090,1.0,66.090,1.0,66.090,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
scaler = load_model['scaler']
X_test_scaled  = scaler.transform(X_test)
X_test_scaled_hide_TX_Fraud  = scaler.transform(X_test_hide_TX_Fraud)
X_test_scaled[0]

array([ 1.70197528,  1.59177252,  2.18147233, -1.39278517,  2.05077018,
       -2.32393305,  2.34965243, -2.65712758,  2.44329133, -0.97951092,
       -0.07583642, -2.33456426, -0.11807996, -3.63132335, -0.14782508])

In [13]:
print(load_model_iso_forest)

{'base_models': {'XGBClassifier': XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.3, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=-1, num_parallel_tree=None, ...), 'LGBMClassifier': LGBMClassifier(max_depth=3, n_jobs=-1, random_state=0, scale_pos_weight=1,
               verbosity=-1), 'CatBoostClassifier': <catboost.core.CatBoostClassifier object at 0x119aade50>, 'HistGra

In [20]:
iso_forest = load_model_iso_forest['isolation_forest']

X_test_scaled_iso  = scaler.transform(X_test)
X_test_scaled_hide_TX_Fraud_iso  = scaler.transform(X_test_hide_TX_Fraud)

X_test_iso  = -iso_forest.score_samples(X_test)
X_test_scaled_hide_TX_Fraud_iso  = scaler.transform(X_test_hide_TX_Fraud)


X_test_scaled_iso[0], X_test_iso

(array([ 1.70197528,  1.59177252,  2.18147233, -1.39278517,  2.05077018,
        -2.32393305,  2.34965243, -2.65712758,  2.44329133, -0.97951092,
        -0.07583642, -2.33456426, -0.11807996, -3.63132335, -0.14782508]),
 array([0.58990163, 0.58990163, 0.58594713, ..., 0.6272043 , 0.61870922,
        0.62297504], shape=(1025,)))

In [15]:
# base_models = load_model['base_models']
# meta_model = load_model['meta_model']

# test_meta = []
# for m in base_models.values():
#     base_test_prob = utils_training.get_predict_proba(m, X_test_scaled)
#     test_meta.append(base_test_prob.reshape(-1, 1))
# X_test_meta = np.hstack(test_meta)
# test_prob = meta_model.predict_proba(X_test_meta)[:, 1]
test_prob = utils_training.get_super_learner_prediction(test_transactions_df, load_model)
test_prob_iso = utils_training.get_super_learner_prediction(test_transactions_df, load_model_iso_forest)

In [16]:
test_prob_hide_TX_Fraud = utils_training.get_super_learner_prediction(test_transactions_df_hide_TX_Fraud, load_model)

In [17]:
transactions_df_scorer = test_transactions_df[['CUSTOMER_ID', 'TX_FRAUD','TX_TIME_DAYS']].copy()
test_auc, test_ap, test_cp = utils_assessment.get_performance_metrics(X_test, y_test, test_prob, transactions_df_scorer)

{'n_train': len(X_test), 'auc': test_auc, 'ap': test_ap, 'cp100': test_cp}

{'n_train': 1025,
 'auc': 0.9137449556093624,
 'ap': 0.8351106304746642,
 'cp100': np.float64(0.024)}

In [19]:
test_auc, test_ap, test_cp = utils_assessment.get_performance_metrics(X_test, y_test, test_prob_iso, transactions_df_scorer)

{'n_train': len(X_test), 'auc': test_auc, 'ap': test_ap, 'cp100': test_cp}

{'n_train': 1025,
 'auc': 0.9112590799031477,
 'ap': 0.8319287733640846,
 'cp100': np.float64(0.024)}

In [18]:
# Create scorer dataframe with TRUE fraud labels for evaluation
transactions_df_scorer_hide_TX_Fraud = test_transactions_df_hide_TX_Fraud[['CUSTOMER_ID', 'TRUE_FRAUD','TX_TIME_DAYS']].copy()
transactions_df_scorer_hide_TX_Fraud.rename(columns={'TRUE_FRAUD': 'TX_FRAUD'}, inplace=True) # needed as card_precision_top_k() determined cp100 based on TX_FRAUD feature.

test_auc, test_ap, test_cp = utils_assessment.get_performance_metrics(X_test_hide_TX_Fraud, y_test_hide_TX_Fraud, test_prob_hide_TX_Fraud, transactions_df_scorer_hide_TX_Fraud)

{'n_test': len(X_test_hide_TX_Fraud), 'auc': test_auc, 'ap': test_ap, 'cp100': test_cp}

{'n_test': 1025,
 'auc': 0.9135754640839386,
 'ap': 0.835035270899062,
 'cp100': np.float64(0.024)}